# Dataset

https://www.kaggle.com/datasets/jensenbaxter/10dataset-text-document-classification

In [1]:
import zipfile
with zipfile.ZipFile('/content/Dataset_text.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/Dataset_text/')

Lectura por lotes desde archivo

In [2]:
from tensorflow import keras
batch_size = 32

train_ds = keras.utils.text_dataset_from_directory(
    "/content/Dataset_text",
    batch_size=batch_size,
    validation_split=0.2,
    label_mode="categorical",
    seed = 2023,
    subset="training",
)

Found 1000 files belonging to 10 classes.
Using 800 files for training.


In [3]:
val_ds = keras.utils.text_dataset_from_directory(
    "/content/Dataset_text",
    batch_size=batch_size,
    validation_split=0.2,
    label_mode="categorical",
    seed = 2023,
    subset="validation",
)

Found 1000 files belonging to 10 classes.
Using 200 files for validation.


Vectorización de datos

In [4]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import TextVectorization,GlobalMaxPooling1D, Dropout, MultiHeadAttention, Dense, LayerNormalization, Embedding

max_length = 600
max_tokens = 20000
text_vectorization = TextVectorization(
    max_tokens=max_tokens,
    output_mode="int",
    output_sequence_length=max_length,    # Limitar la entrada a 600 palabras
)

# Extraer solo el texto (features) de los datos de entrada para calcular el vocabulario
text_only_train_ds = train_ds.map(lambda x, y: x)

text_vectorization.adapt(text_only_train_ds)

int_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

int_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

# Transformer encoder

In [5]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [Dense(dense_dim, activation="relu"),
             Dense(embed_dim),]
        )
        self.layernorm_1 = LayerNormalization()
        self.layernorm_2 = LayerNormalization()

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
        attention_output = self.attention(
            inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

# Modelo (Self attention)

In [6]:
vocab_size = 20000
embed_dim = 256
num_heads = 2
dense_dim = 32

inputs = keras.Input(shape=(None,), dtype="int64")
x = Embedding(vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
x = GlobalMaxPooling1D()(x)
x = Dropout(0.5)(x)
outputs = Dense(10, activation="softmax")(x)

model = keras.Model(inputs, outputs)

model.compile(optimizer="rmsprop",
              loss="categorical_crossentropy",
              metrics=["accuracy"])

model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, None)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding (Embedding)                │ (None, None, 256)           │       5,120,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ transformer_encoder                  │ (None, None, 256)           │         543,776 │
│ (TransformerEncoder)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_max_pooling1d                 │ (None, 256)                 │               0 │
│ (GlobalMaxPooling1D)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 10)                  │           2,570 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,666,346 (21.62 MB)

 Trainable params: 5,666,346 (21.62 MB)

 Non-trainable params: 0 (0.00 B)

# Entrenamiento

In [7]:
"""
callbacks = [
    keras.callbacks.ModelCheckpoint("transformer_encoder.h5",
                                    save_best_only=True)
]
"""

model.fit(int_train_ds,
          validation_data=int_val_ds,
          epochs=20,
          #callbacks=callbacks
          )

"""
model = keras.models.load_model(
    "transformer_encoder.h5",
    custom_objects={"TransformerEncoder": TransformerEncoder}
    )

#print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")
"""

Epoch 1/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 15s 301ms/step - accuracy: 0.1489 - loss: 4.3818 - val_accuracy: 0.1900 - val_loss: 2.1360
Epoch 2/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 84ms/step - accuracy: 0.1840 - loss: 2.7367 - val_accuracy: 0.3200 - val_loss: 1.7924
Epoch 3/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 82ms/step - accuracy: 0.3097 - loss: 2.2369 - val_accuracy: 0.4450 - val_loss: 1.6254
Epoch 4/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 81ms/step - accuracy: 0.5015 - loss: 1.5455 - val_accuracy: 0.8000 - val_loss: 0.8498
Epoch 5/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 85ms/step - accuracy: 0.6610 - loss: 1.0358 - val_accuracy: 0.8900 - val_loss: 0.3998
Epoch 6/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 84ms/step - accuracy: 0.8235 - loss: 0.5256 - val_accuracy: 0.9150 - val_loss: 0.3468
Epoch 7/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 86ms/step - accuracy: 0.8757 - loss: 0.3623 - val_accuracy: 0.9050 - val_loss: 0.3014
Epoch 8/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 85ms/step - accuracy: 0.9214 - loss: 0.2600 - val_accuracy: 0.9300 -

'\nmodel = keras.models.load_model(\n    "transformer_encoder.h5",\n    custom_objects={"TransformerEncoder": TransformerEncoder}\n    )\n\n#print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")\n'